In [1]:
from abc import ABC, abstractmethod
import numpy as np
import matplotlib.pyplot as plt

### GridWorld using Value Iteration: In this example, we will use value iteration to obtain the Q/value -function for the GridWorld problem and extract a policy

In [3]:
# first, define an interface for an MDP
class MDP:
    @abstractmethod
    def get_states(self):
        pass   

    @abstractmethod
    def get_initial_state(self):
        pass

    @abstractmethod
    def get_terminal_states(self):
        pass    

    @abstractmethod
    def get_actions(self, state):
        pass

    @abstractmethod
    def get_transitions(self, state, action):
        pass

    @abstractmethod
    def get_rewards(self, state, action, next_state):
        pass

    @abstractmethod
    def is_terminal(self, state):
        pass

    @abstractmethod
    def get_discount_factor(self):
        pass
    
    @abstractmethod
    def execute(self):
        pass




# now, implement the gridworld problem MDP
class GridWorld(MDP):

    def __init__(self, discount_factor=0.9) :

        # initialise the set of all possible states, in this case tuples (x,y) of all grid cells, excluding walls
        # will use the example from lectures and tute week 7
        self.width = 4
        self.height = 3
        self.walls = [(1,1)]
        self.states = []
        for x in range(self.width):
            for y in range(self.height):
                if (x,y) not in self.walls:
                   self.states.append((x,y))

        #print(f"States: {self.states}")

        # specify terminal states
        self.terminal_states=[(3,1),(3,2)] 

        # create a dummy terminal state which is the successor to all terminal states
        self.exit = (-1,-1)

        # specify initial state
        self.initial_state = (0,0)

        # specify probability of splipping
        self.noise = 0.1

        # specify/enumerate the actions
        self.terminate = 0
        self.up = 1
        self.down = 2
        self.left = 3
        self.right = 4
        
        # set the discount factor
        self.gamma = discount_factor

        # specify rewards
        self.rewards =  {(3,1) : -1, (3,2) : 1}

        # specify action cost 
        self.action_cost = 0.0

        # empty list for storing the discounted reward at each step opf the episode
        self.episode_discounted_rewards = []

        # initialize Q-function and value function
        self.Q = {}
        self.V = {}
        self.Vtemp = {}
        for state in (self.states + [self.exit]):
            self.Q[(state, self.up)] = 0.0
            self.Q[(state, self.down)] = 0.0
            self.Q[(state, self.left)] = 0.0
            self.Q[(state, self.right)] = 0.0
            self.V[state] = 0.0
            self.Vtemp[state] = 0.0


    def get_states(self):
        return self.states   


    def get_initial_state(self):
        return self.initial_state


    def get_terminal_states(self):
        return self.terminal_states


    def get_actions(self, state):
        if state not in self.terminal_states:
            actions = [self.up, self.down, self.left, self.right]
        else:
            # for terminal states, the only valid action is to 'terminate' the episode
            actions = [self.terminate]
        return actions        

    
    # for given state-action pair, returns possible successor states along with their corresponding transition probabilities
    def get_transitions(self, state, action):
        
        # if we're in an terminal state, then the only allowed action is to transition into the 'exit' state which terminates the episode
        if state in self.terminal_states:
            if action == self.terminate:
                return [(self.exit, 1.0)]
            else:
                return []

        # probability of not slipping
        straight = 1 - 2*self.noise

        transitions = []
        (x,y) = state

        if action == self.up:
            transitions.append(self.valid_add(state, (x,y+1), straight))
            transitions.append(self.valid_add(state, (x-1,y), self.noise))
            transitions.append(self.valid_add(state, (x+1,y), self.noise))

        elif action == self.down:
            transitions.append(self.valid_add(state, (x,y-1), straight))
            transitions.append(self.valid_add(state, (x-1,y), self.noise))
            transitions.append(self.valid_add(state, (x+1,y), self.noise))

        elif action == self.left:
            transitions.append(self.valid_add(state, (x-1,y), straight))
            transitions.append(self.valid_add(state, (x,y-1), self.noise))
            transitions.append(self.valid_add(state, (x,y+1), self.noise))

        elif action == self.right:
            transitions.append(self.valid_add(state, (x+1,y), straight))
            transitions.append(self.valid_add(state, (x,y-1), self.noise))
            transitions.append(self.valid_add(state, (x,y+1), self.noise))

        return transitions
    

    def get_rewards(self, state, action, next_state):
        if (state in self.terminal_states) and (next_state == self.exit):
            reward = self.rewards[state]
        else:
            reward = self.action_cost

        # store discounted reward for the step
        step = len(self.episode_discounted_rewards)
        self.episode_discounted_rewards.append(reward * (self.gamma**step))

        return reward    


    def is_exit(self, state):
        return (state == self.exit)    


    def get_discount_factor(self):
        return self.gamma  

    
    def execute(self, state, action):
        # get all transitions
        transitions = self.get_transitions(state, action)
        states = [tr[0] for tr in transitions]
        probs = [tr[1] for tr in transitions]
        # sample from the transitions to get the next state state
        next_state = np.random.choice(states, size=1, p=probs)
        reward = self.get_rewards(state, action, next_state)
        return (next_state, reward)
    

    def valid_add(self, state, next_state, probability):

        # check if next state is a wall
        if next_state in self.walls:
            return (state, probability)

        # check if next state is off grid
        (x,y) = next_state
        if (x>=0 and x<=(self.width-1) and y>=0 and y<=(self.height-1)):
            return (next_state, probability)
        else:
            return (state, probability)


    def update_Q(self, state, action, Qnew):
        self.Q[(state, action)] = Qnew
    
    
    def update_V(self):
        # copy from Vtemp
        for state in self.states:
            self.V[state] = self.Vtemp[state]


    def extract_policy(self):
        # extract a policy using the Q function
        policy = {}
        Qs = {}
        for state in self.states:
            Qs['up'] = self.Q[(state, self.up)] 
            Qs['down'] = self.Q[(state, self.down)]
            Qs['left'] = self.Q[(state, self.left)] 
            Qs['right'] = self.Q[(state, self.right)]         
            policy[state] = max(Qs, key=lambda k:Qs[k])
            
        return policy



In [12]:
# implement the value iteration loop

theta = 0.001 # threshold

def value_iteration(grid_world_mdp, num_iters):
    states = grid_world_mdp.get_states()    
    gamma = grid_world_mdp.get_discount_factor()
    for i in range(num_iters):
        print(f"Iteration# {i}")
        d = 0.0 
        for state in states:
            #print(f"state: {state}")
            actions = grid_world_mdp.get_actions(state)
            Qsa = [] 
            for action in actions:
                #print(f"action: {action}")
                # get the transitions
                transitions = grid_world_mdp.get_transitions(state, action)
                #print(f"transitions: {transitions}")
                # compute Q value
                Qnew = 0.0
                for tr in transitions:
                    (next_state, p) = tr
                    Qnew += p * (grid_world_mdp.get_rewards(state, action, next_state) + gamma * grid_world_mdp.V[next_state])
                grid_world_mdp.update_Q(state, action, Qnew)
                Qsa.append(Qnew)
            Vnew = max(Qsa)    
            grid_world_mdp.Vtemp[state] = Vnew  
            # update delta        
            d = max(d, abs(Vnew - grid_world_mdp.V[state]))
        
        # update the value function
        grid_world_mdp.update_V()

        # diagnostic updated Q values in cell (0,0)
        #print(f"Cell(0,0): V = {grid_world_mdp.V[(0,0)]}, Qup = {grid_world_mdp.Q[((0,0),1)]}, Qdown = {grid_world_mdp.Q[((0,0),2)]}, Qleft = {grid_world_mdp.Q[((0,0),3)]}, Qright = {grid_world_mdp.Q[((0,0),4)]}")
        #print(f"Cell(2,2): V = {grid_world_mdp.V[(2,2)]} ,Qup = {grid_world_mdp.Q[((2,2),1)]}, Qdown = {grid_world_mdp.Q[((2,2),2)]}, Qleft = {grid_world_mdp.Q[((2,2),3)]}, Qright = {grid_world_mdp.Q[((2,2),4)]}")

        print("-----------------------")
        for y in range(grid_world_mdp.height-1, -1, -1):
            for x in range(grid_world_mdp.width):
                if (x,y) in grid_world_mdp.V:
                    print(f"{grid_world_mdp.V[(x,y)]: 0.2f}", end=' ')
                else:
                    print(f"{0.0: 0.2f}", end=' ')
            print("")       
        print("-----------------------")


        # stop value iteration if delta falls below threshold    
        if d <= theta:
            break    


In [13]:
# instantiate grid world mdp object
gw = GridWorld(discount_factor=0.9)

value_iteration(gw, num_iters=50)

Iteration# 0
-----------------------
 0.00  0.00  0.00  1.00 
 0.00  0.00  0.00 -1.00 
 0.00  0.00  0.00  0.00 
-----------------------
Iteration# 1
-----------------------
 0.00  0.00  0.72  1.00 
 0.00  0.00  0.00 -1.00 
 0.00  0.00  0.00  0.00 
-----------------------
Iteration# 2
-----------------------
 0.00  0.52  0.78  1.00 
 0.00  0.00  0.43 -1.00 
 0.00  0.00  0.00  0.00 
-----------------------
Iteration# 3
-----------------------
 0.37  0.66  0.83  1.00 
 0.00  0.00  0.51 -1.00 
 0.00  0.00  0.31  0.00 
-----------------------
Iteration# 4
-----------------------
 0.51  0.72  0.84  1.00 
 0.27  0.00  0.55 -1.00 
 0.00  0.22  0.37  0.13 
-----------------------
Iteration# 5
-----------------------
 0.59  0.73  0.85  1.00 
 0.41  0.00  0.57 -1.00 
 0.21  0.31  0.43  0.19 
-----------------------
Iteration# 6
-----------------------
 0.62  0.74  0.85  1.00 
 0.50  0.00  0.57 -1.00 
 0.34  0.36  0.45  0.24 
-----------------------
Iteration# 7
-----------------------
 0.63  0.74

In [15]:
# policy extraction
pi = gw.extract_policy()


In [11]:
print("-----------------------")
for y in range(gw.height-1, -1, -1):
    for x in range(gw.width):
        if (x,y) in pi:
            print(f"{pi[(x,y)]}", end=' ')
        else:
            print(f"{'None'}", end=' ')
    print("")       
print("-----------------------")

-----------------------
right right right up 
up None up up 
up left up left 
-----------------------


### Q-Learning: For model free MDPs, we cxannot use value iteration. Instead, we can bootstrap from an estimate of the value function to update Q values. 

In [16]:
# create a Q-learner class
class QLearner:
    def __init__(self, mdp, alpha=0.1) :
        self.mdp = mdp
        self.alpha = alpha
        self.epsilon = 0.1
        np.random.seed(2)


    # epsilon-greedy multi-arm bandit
    def bandit(self, state):    
        # get all available actions
        actions = self.mdp.get_actions(state)
        randnum = np.random.random()
        # select next actions via exploration-exploitation
        if randnum < self.epsilon:
            action = np.random.sample(actions, size=1)
        else:
            # get Q values
            Qsa = {action:self.mdp.Q[(state, action)] for action in actions}
            # argmax to find best action
            action = max(Qsa, key=Qsa.get)        
        return action


    # Q-learning update
    def get_delta(self, reward, Qold, state, next_state, next_action):
        # get estimated value for next state
        aprime = self.mdp.get_actions(next_state)
        Vsprime = max([self.mdp.Q[(next_state, action)] for action in aprime])
        delta = reward + self.mdp.gamma * Vsprime - Qold
        return self.alpha * delta  


    # Q-learner training loop
    def train(self, episodes=10):

        for i in range(episodes):
            # get initial state for the episode
            state = self.mdp.get_initial_state()
            # select an action using bandit
            action = self.bandit(state)
            # repeat until terminal state is reached
            while not self.mdp.is_exit(state):
                (next_state, reward) = self.mdp.execute(state, action)
                next_action = self.bandit(next_state)
                # update q value
                Qold = self.mdp.Q[(state, action)]

SyntaxError: unexpected EOF while parsing (2139099261.py, line 9)